In [35]:
def plot_gals(gals, npix=200, fn_save=None, ioff=True,
             do9plots = False, add_cb=False, i_lambda=0, **kwargs):

    import matplotlib.pyplot as plt
    from matplotlib import ticker
    from draw import pp
    from matplotlib.colors import LogNorm
    import matplotlib as mpl
    import numpy as np
    if ioff:
        plt.ioff()
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    from matplotlib.ticker import MaxNLocator
    nullfmt = plt.NullFormatter()

    def fmt(x, pos):
        a, b = '{:.2e}'.format(x).split('e')
        b = int(b)
        return r'${} \times 10^{{{}}}$'.format(a, b)


    def _close_to(values, v_ref):
        """
            returns the index of the value which is close to the v_ref.
        """
        ind = 0
        good=values[ind]
        for i, vv in enumerate(values):
            if abs(vv - v_ref) < good:
                good = vv
                ind = i

        return ind

    fig, axs = plt.subplots(4,3)
    fig.set_size_inches(6.5, 8)
    plt.subplots_adjust(wspace=0.02, hspace=0.01, left=0.14, right=0.85)
    for icol, axs_col in enumerate(axs.T):
        self = gals[icol]
        rgal_to_reff = self.meta.Rgal_to_reff
        
        axden=axs_col[0]
        axvel=axs_col[1]
        axsig=axs_col[2]
        axprf=axs_col[3]

        
    # Stellar particle density map
        """
            1st plot. - Stellar density map. 
        """
        ax = axden
        # if hist=True, use histogram instead of custom CIC.
        img = pp.part2den(self.star, self.info, npix=npix, hist=True)
        img.data = np.nan_to_num(img.data) + 1
        im = axden.imshow(img.data, cmap=plt.get_cmap('bone'),
                       norm=LogNorm(vmin=1e6))
        

    #   Todo:
    #   If rgal_to_reff is not too large, say 10. xticks are too dense. 
    #   If 8, or 9 is given, use 4, if 10, use 5, and so on.. 
    #   Utilize _close_to function.
        if icol==0:
            ##ax.set_title("stellar surface density")
            ax.set_title(r"$\Sigma_{*}$", fontsize=18,
                        rotation='vertical',x=-0.2,y=0.45)

        if icol==2:
            axins = inset_axes(ax,
                               width="5%", # width = 10% of parent_bbox width
                               height="90%", # height : 50%
                               loc=3,
                               bbox_to_anchor=(1.05, 0, 1, 1),
                               bbox_transform=ax.transAxes,
                               borderpad=0,
                               )                
            cb = plt.colorbar(im, cax=axins,
                              orientation="vertical") # needs a mappable object.
            cb.set_label(r'$\rm{M_{\odot}\,kpc^{-2}}$', labelpad=-1)
            #tick_locator = ticker.MaxNLocator(nbins=3)
            #cb.locator = tick_locator
            #cb.update_ticks()  
            
            
        ax.xaxis.set_major_formatter(nullfmt)
        
        patch = mpl.patches.FancyArrowPatch(
                        (0, npix*1.05), (npix, npix*1.05),
                        arrowstyle='<->',
                        edgecolor="k",
                        fill=False,
                        joinstyle=['miter', 'round', 'bevel'][0],
                        alpha = 1,
                        arrow_transmuter=None, 
                        mutation_aspect=None,
                        dpi_cor=1.0)
        ax.add_patch(patch)
        ax.text(0.5, 1.25,
                "{:d}".format(self.meta.id),
                horizontalalignment='center',
                transform=ax.transAxes
                )
        ax.text(0.5, 1.1,
                r"$R_{gal}$" + "= {:.1f}kpc".format(self.meta.rgal),
                horizontalalignment='center',
                transform=ax.transAxes
                )
        ax.axis('off')

        
    # sigma map
        if hasattr(self, "sigmap"):
            l_range = self.meta.reff * self.meta.rscale_lambda
            ax = axsig
            self.sigmap[self.sigmap == 0] = np.nan
            im = ax.imshow(self.sigmap, vmin=0, vmax = 300, cmap='RdYlBu_r')
            ax.contour(np.log10(self.mmap), 6, colors="k")
            
            
            if icol==0:
                ax.set_title(r"$\sigma$", fontsize=18,
                        rotation='vertical',x=-0.2,y=0.45)

            ax.yaxis.set_major_formatter(nullfmt)
            if icol==2:
                axins = inset_axes(ax,
                                   width="5%", # width = 10% of parent_bbox width
                                   height="90%", # height : 50%
                                   loc=3,
                                   bbox_to_anchor=(1.05, 0, 1, 1),
                                   bbox_transform=ax.transAxes,
                                   borderpad=0,
                                  )                
                cb = plt.colorbar(im, cax=axins,
                                  orientation="vertical") # needs a mappable object.
                cb.set_label(r'$\rm{km\,s^{-1}}$', labelpad=-1)
                tick_locator = ticker.MaxNLocator(nbins=3)
                cb.locator = tick_locator
                cb.update_ticks()  
            
            if False:
                ax.set_ylabel("\rm{[kpc]}")
                npix_this = len(self.sigmap)
                ax.set_yticks(np.linspace(npix_this/8, 7*npix_this/8, 3))
                yticks = ["{:.1f}".format(y)for y in np.linspace(-self.meta.rgal*3/4, self.meta.rgal*3/4, num=3)]
                ax.set_yticklabels(yticks)        

    # velocity map
        if self.vmap is not None:
            l_range = self.meta.reff * self.meta.rscale_lambda
            ax = axvel
            self.vmap[self.vmap == 0] = np.nan
            im = ax.imshow(self.vmap, vmin=-100, vmax = 100, cmap='RdYlBu_r')
            ax.contour(np.log10(self.mmap), 6, colors="k")
            
            if icol==0:
                ax.set_title(r"$v$", fontsize=18,
                            rotation='vertical',x=-0.2, y=0.45)
            ax.yaxis.set_major_formatter(nullfmt)

            if icol==2:                
                               
                axins = inset_axes(ax,
                   width="5%", # width = 10% of parent_bbox width
                   height="90%", # height : 50%
                   loc=3,
                   bbox_to_anchor=(1.05, 0, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )                
                cb = plt.colorbar(im, cax=axins,                                  
                                  orientation="vertical") # needs a mappable object.
                cb.set_label(r'$\rm{km\,s^{-1}}$', labelpad=-8)
                tick_locator = ticker.MaxNLocator(nbins=3)
                cb.locator = tick_locator
                cb.update_ticks()
                            

    # Lambda_r plot
        if self.meta.lambda_result_list is not None:
            ll = self.meta.lambda_result_list[i_lambda]
            ax = axprf
            ax.plot(ll) # ~ 1 * Reff
            ax.text(0.1* len(ll), 0.7, r"$log_{10}(M_{*}) = $" + "{:.1f}".format(np.log10(self.meta.mstar)))# + r"$M_{\odot}$")
            ax.set_ylim([0,0.8])
            
            if icol == 0:
                ax.set_ylabel(r'$\lambda_{R}$', fontsize=18)
                ax.yaxis.set_major_locator(MaxNLocator(8, prune='upper'))
            else:
                ax.yaxis.set_major_formatter(nullfmt)

            unit = len(ll)/self.meta.rscale_lambda # number of points per 1Reff
            xticks = [0]
            reffs=[0]
            for i in range(int(self.meta.rscale_lambda) + 1): # If lambda_r is calculated furthrer -
                xticks.append(unit * (i+1))
                reffs.append((i+1))
                ax.set_xticks(xticks)
                xticks_label = ["{:.1f}".format(rf) for rf in reffs]
                ax.set_xticklabels(xticks_label)
                ax.set_xlabel("["+ r'$R/R_{\rm{eff}}$'+"]")
            ax.set_xlim([0, 20])
            ax.xaxis.set_major_locator(MaxNLocator(4, prune='upper'))

    for icol, axs_col in enumerate(axs.T):
        for ax in axs_col[:3]:
            ax.xaxis.set_ticks_position('none') 
            ax.yaxis.set_ticks_position('none') 
            ax.xaxis.set_major_formatter(nullfmt)
            ax.yaxis.set_major_formatter(nullfmt)
            
    if fn_save is None:
        fn_save = "/home/hoseung/Work/data/figs/Odd_gal_plot" + str(self.meta.id).zfill(5)

    plt.savefig(fn_save + ".png", dpi=200)
    plt.savefig(fn_save + ".pdf", dpi=200)
    #plt.show()


## View details of odd galaxies.

160500811,  160500745, 3566300002, 2917600065, 2917600001,
       1000200001, 3999000032, 3999000015, 3999000001, 3641300444,
       3641500003

In [52]:
import matplotlib.pyplot as plt

import numpy as np
import collections
from galaxymodule import galaxy
import pickle
import load
from load.hydro import Hydro
#import pandas as pd

from analysis.cal_lambda import *

#idgal_list=[40, 15, 45]
def worker(gals, hals, out_q, info, inds,
           dump_gal=False,
           reorient=False,
           galaxy_plot=False,
           galaxy_plot_dir='galaxy_plot/',
           region_plot=False,
           wdir='./',
           with_DM=True,
           with_cell=True,
           mk_gal_params={},
           cal_lambda_params={},
           rscale_extract_cell=1.0,
           **kwargs):

    import galaxy.make_gal as mkg 
    from galaxymodule import galaxy

    nout = info.nout
    if type(inds) == int:
        inds = [inds]
    
    gals_done = []
    
    print("AAAAAAAAAAAAAAAAAAAA",inds)
    
    for i in inds:
        print("In worker, inds = ",i)
        gg = gals.data[i]
        hh = hals.data[i]

        galid = gg['id']
        halid = hh['id']
        gm = load.rd_GM.rd_gal(info.nout, galid, wdir=wdir)
        print('!!!! mima vx in gm', min(gm.star['vx']), max(gm.star['vx']))

        if with_cell:
            #gm.cell = load.rd_GM.rd_cell(info.nout, galid, wdir=wdir)
            gm.cell = extract_cell(hh, info, wdir, rscale=rscale_extract_cell)
        else:
            gm.cell = None
        if with_DM:
            if halid > 0 :
                gm.dm = load.rd_GM.rd_dm(info.nout, halid, wdir=wdir)
            else:
                gm.dm = extract_dm(hh, info, wdir)
                # The   se are not in the final units that I want,
                # but in the same units as GM outputs to make later conversion steps simpler.
                gm.dm['x'] = (gm.dm['x'] - 0.5) * info.pboxsize
                gm.dm['y'] = (gm.dm['y'] - 0.5) * info.pboxsize
                gm.dm['z'] = (gm.dm['z'] - 0.5) * info.pboxsize
                gm.dm['vx'] *=info.kms
                gm.dm['vy'] *=info.kms
                gm.dm['vz'] *=info.kms
                gm.dm['m'] * info.msun/1e11
        else:
            gm.dm = None

        gal = galaxy.Galaxy(halo=gg, info=info)
        good_gal = gal.mk_gal(gm.star, gm.dm, gm.cell,
                              unit_conversion="GM",
                              verbose=False)
        if with_cell:
            print("GAS MASS in solar mass", gal.meta.mgas)

        galid = gal.meta.id

        #do other things
        if not good_gal:
            print(galid, " Not a good galaxy")
            #out_q.put(gal.meta)
        else:
            print("galaxy {} is made \n".format(galid))
            gal.cal_lambda_r_eps(**cal_lambda_params)

            if galaxy_plot:
                gal.plot_gal(fn_save = galaxy_plot_dir + str(nout).zfill(3)                                      + "_" + str(galid) + ".png", ioff=True)

            gal.meta.__dict__['idx'] = hals.data['idx'][i]
            gal.meta.__dict__['rhalo'] = hals.data['rvir'][i]

            print("   lambda calculation done. ID, IDx", galid, gal.meta.idx)

            if reorient:
                reori_pops = ["star"]
                if with_cell:
                    reori_pops.append("cell")
                if with_DM:
                    reori_pops.append("dm")

                gal.cal_norm_vec(["star"], dest=[0.,1.,0.])
                gal.cal_rotation_matrix(dest = [0., 1., 0.])
                gal.reorient(dest=[0., 1., 0.], pop_nvec = ['star'],
                             pops=reori_pops, verbose=True)
                lambdas = gal.cal_lambda_r_eps(**cal_lambda_params)
                                #npix_per_reff=npix_lambda,
                                #rscale=rscale_lambda, method='ellip')
# Because lambda is measured for ALL stars,
# B/T must also be measured for ALL stars, not only for bound stars.
                gal.meta.lambda_arr2, gal.meta.lambda_arr2h, gal.meta.lambda_arr2q = lambdas[0]
                gal.meta.lambda_r2,   gal.meta.lambda_r2h  , gal.meta.lambda_r2q   = lambdas[1]
                if galaxy_plot:
                    gal.plot_gal(fn_save = galaxy_plot_dir + str(nout).zfill(3)                                          + "_" + str(galid) + "_reori.png", ioff=True)
        out_q.put(gal.meta.__dict__)
        #print("where are you gone, dict?", out_q.get())

        gals_done.append(gal)
    return gals_done

In [66]:
def do_main(wdir='./',
         ncore=1,
         nout_ini=37,
         nout_end=187,
         read_halo_list=True,
         out_dir="",
         cat_suffix="",
         r_cluster_scale=2.9,
         is_gal = True,
         n_pseudo=60,
         w_dump_gal=False,
         w_reorient=False,
         w_galaxy_plot=True,
         w_galaxy_plot_dir= 'galaxy_plot/',
         w_region_plot=False,
         w_wdir='./',
         w_with_DM=False,
         w_with_cell=False,
         save_result=False,
         idgal_list=[],
         ):
    from tree import treemodule
    #import os
    import pickle
    import tree.ctutils as ctu

    verbose=False
    log_run_detail = False
    if w_galaxy_plot:
        import os
        if not os.path.isdir(w_galaxy_plot_dir):
            os.mkdir(w_galaxy_plot_dir)
    # worker options
    worker_params = {"dump_gal":       w_dump_gal,
                     "reorient":       w_reorient,
                     "galaxy_plot":    w_galaxy_plot,
                     "galaxy_plot_dir":out_dir + w_galaxy_plot_dir,
                     "region_plot":    w_region_plot,
                     "wdir":           w_wdir,
                     "with_DM":        w_with_DM,
                     "with_cell":      w_with_cell
                    }
    multi = False
    if ncore > 1: multi=True
    if multi:
        import multiprocessing as mp
    else:
        import queue

    dir_cat = out_dir

    #dump_gal = True
    nout_complete = 87 # at least complete up to z = 1

    masscut_a = 1256366362.16
    masscut_b = -20583566.5218

    # optional parameters ----------------------------------------------------
    #lambda_method = 'ellip' 
    #galaxy_plot = False
    #reorient = True
    #region_plot = False

    if out_dir == "":
        out_dir = "out_default/"

    if out_dir[-1] != '/':
        out_dir = out_dir + '/'
        # need a tailing slash to be joined with sub-directories

    out_base = wdir + out_dir

    # 27 : z=4;  37 : z=3;  20 : ~ z=5
    if nout_ini == "":
        nout_ini = 37
    else:
        nout_ini = int(nout_ini)

    if nout_end == "":
        nout_end = 187
    else:
        nout_end = int(nout_end)

    nout_fi = 187
    nouts = range(nout_fi, nout_ini -1, -1)
    nouts_dump_gal = []
    try: nouts_dump_gal
    except NameError: nouts_dump_gal = None
    #----------------------------------------------------------------------

    # Only galaxies above this stellar mass at the final snapshot are considered.
    mstar_min = 5e9
    mk_gal_rscale = 1.1 # unit of Rvir,galaxy
    #r_cluster_scale = 2.9 # maximum radius inside which galaxies are searched for
    npix=800
    lmax = 19
    ptypes=["star id pos mass vel time metal", "dm id pos mass vel"]

    voronoi_dict = dict(targetSN=15, plot=False, quiet=True)
    # voronoi tesselation need many points,
    # always use voronoi with pseudo particle option
    cal_lambda_params = dict(npix_per_reff=5,
                             rscale=3.0,
                             method='ellip',
                             n_pseudo=n_pseudo,
                             verbose=False,
                             voronoi=None,#voronoi_dict,
                             mge_interpol = True)

    ## halo part -----------------------------------------------------------
    out_dir_cat = wdir + dir_cat + '/'

    # Load complete tree -----------------------------------------------------
    if is_gal:
        # Galaxy tree
        tree_path = 'GalaxyMaker/Trees/'
        m_halo_min = 5e9 # minimum galaxy mass above which galaxies are searched for. 
    else:
        # halo tree
        tree_path = 'halo/Trees/'
        m_halo_min = 2e10 # minimum halo mass. 
##----------------------------------------------------------------------------------
    if read_halo_list:
        try:
            prg_only_tree = pickle.load(open(wdir + "prg_only_tree.pickle", 'rb'))
            read_halo_list = True
            print("loading pickled halo list done:")
        except:
            read_halo_list = False

    if not read_halo_list:
        try:
            alltrees = pickle.load(open(wdir + tree_path + "extended_tree.pickle", "rb" ))
            print("Loaded an extended tree")
        except:
            alltrees = treemodule.CTree()
            alltrees.load(filename= wdir + tree_path + 'tree_0_0_0.dat')
            # Fix nout -----------------------------------------------------
            nout_max = alltrees.data['nout'].max()
            alltrees.data['nout'] += nout_fi - nout_max
            print("------ NOUT fixed")
            alltrees.data = ctu.augment_tree(alltrees.data, wdir, is_gal=is_gal)
            print("------ tree data extended")
            pickle.dump(alltrees, open(wdir + tree_path + "extended_tree.pickle", "wb" ))
        # Reading tree done
        info = load.info.Info(nout=nout_fi, base=wdir, load=True)
        prg_only_tree = get_sample_tree(alltrees, info,
                        wdir=wdir,
                        nout_ini=nout_ini,
                        nout_fi =nout_fi,
                        is_gal = is_gal,
                        r_cluster_scale = r_cluster_scale,
                        m_halo_min = m_halo_min,
                        nout_complete = nout_complete)
        pickle.dump(prg_only_tree, open(wdir + 'prg_only_tree.pickle', 'wb'))

##----------------------------------------------------------------------------------
    if log_run_detail:
        with open(wdir + 'lambda_mp_status.txt', 'w') as f:
            f.write("mstar_min = " + str(mstar_min) + "\n")
            f.write("Rscale cluster : " + str(r_cluster_scale))
            f.write("Rscale mk_gal : " + str(mk_gal_rscale))
            f.write("npix : " + str(npix))
            save_dict_scalar(cal_lambda_params, f, "\n")
            #f.write("Rscale lambda calculation : " + str(rscale_lambda))
            #f.write("npix per 1Reff for lambda calculation : " + str(npix_lambda))
            f.write("ptypes : \n")
            for i in ptypes:
                f.write("  " + str(i) + "\n")
##----------------------------------------------------------------------------------
    for nout in nouts:
        print(nout, nout_fi)

        snout = str(nout)
        if nout > nout_end:
            continue
        if nout in nouts_dump_gal:
            dump_gal = True
            print("DUMP TRUE___________________________________")
        else:
            dump_gal = False

        fcat = out_dir_cat +"catalog" + snout + cat_suffix +".pickle"
        galaxy_plot_dir = out_base + 'galaxy_plot' + snout + '/'
        #if not os.path.isdir(galaxy_plot_dir):
        #    os.mkdir(galaxy_plot_dir)

##----------------------------------------------------------------------------------
        info = load.info.Info(nout=nout, base=wdir, load=True)
        if nout < 187:
            mstar_min = 2 * get_mstar_min(info.aexp)
        print("Mstar now, min: {:.2e} {:.2e}".format(2 * (masscut_a * info.aexp + masscut_b), mstar_min))

        allgal, allhal = get_sample_gal(wdir, nout, info, prg_only_tree, mstar_min)
##----------------------------------------------------------------------------------

        #nh = len(allgal.data)
        nh = len(idgal_list)
        print("Total # galaxies to analyze",nh)
        print("# complete-tree galaxies",sum(allhal.data['idx'] > 0))
        print("# non-complete-tree galaxies",sum(allhal.data['idx'] < 0))

        mk_gal_params = dict(verbose=verbose,
                             mstar_min=mstar_min,
                             unit_conversion="GM")

        inds = mtc.match_list_ind(allgal.data["id"], idgal_list)
        out_q = queue.Queue()
        
        gals = worker(allgal, allhal, out_q, info, inds,
               mk_gal_params=mk_gal_params,
               cal_lambda_params=cal_lambda_params,
               **worker_params)

        print("----------Done---------")
        
        
        if save_result:
            dictout=[]
            try:
                #if not os.path.isdir(out_dir_cat):
                #    os.mkdir(out_dir_cat)
                f = open(out_dir_cat + 'galaxies' + snout + '.txt', 'w')
                #write header
                dd =  out_q.get(timeout=0.1)
                for key in sorted(dd.keys()):
                    if isinstance(dd[key], collections.Iterable):
                        continue
                    else:
                        f.write(str(key) + ", ")
                        f.write("\n")
                save_dict_scalar(dd, f, ", ")
                dictout.append(dd)
                for i in range(nh-1):
                    try:
                        dd =  out_q.get(timeout=0.1)
                        if dd['id'] == 0:
                            continue
                        save_dict_scalar(dd, f, ", ")
                        dictout.append(dd)
                    except:
                        continue
                f.close()
                print("Text file written")
            except:
                print("No filename is given.\n ")

        
            with open(fcat, 'wb') as f:
                pickle.dump(dictout, f)

        s = 0
        # minimum stellar mass check only for the final snapshot galaxies,
        # No more mstar_min test.
        print("------------------ \n")
        return gals
        #print("main loop took ", time.time() - t0, "seconds")

In [72]:
wdir = "/home/hoseung/Work/data/36415/"
gals = do_main(wdir=wdir, w_wdir=wdir, w_galaxy_plot=True,
               w_galaxy_plot_dir= wdir +'galaxy_plot/',
                 idgal_list=np.array([3])))

wdir = "/home/hoseung/Work/data/10002/"
gals.extend(do_main(wdir=wdir, w_wdir=wdir, w_galaxy_plot=True,
               w_galaxy_plot_dir= wdir +'galaxy_plot/',
               idgal_list=np.array([1])))

wdir = "/home/hoseung/Work/data/36413/"
gals.extend(do_main(wdir=wdir, w_wdir=wdir, w_galaxy_plot=True,
               w_galaxy_plot_dir= wdir +'galaxy_plot/',
               idgal_list=np.array([444])))

plot_gals(gals, npix=60)

loading pickled halo list done:
187 187
Mstar now, min: 2.47e+09 5.00e+09
[Halo.load_info] loading info
[Halo.load_info] nout = 187, base =/home/hoseung/Work/data/36415/
/home/hoseung/Work/data/36415/halo/DM/tree_bricks187
[Halo.load_info] loading info
[Halo.load_info] nout = 187, base =/home/hoseung/Work/data/36415/
(catalogue) # galaxies more massive than 5.00e+09 at nout = 187
 Originally the tree selected: 85
Total set length: 105
length of the original allgal: 105
Before associate_gal_hal,length of the original allhal: 2854
Now, 105
Total # galaxies to analyze 1
# complete-tree galaxies 85
# non-complete-tree galaxies 20
AAAAAAAAAAAAAAAAAAAA [2]
In worker, inds =  2
[rd_GM.rd_gal] fname= /home/hoseung/Work/data/36415/GalaxyMaker/GAL_00187/gal_stars_0000003
!!!! mima vx in gm -1190.76647949 1008.30047607


/home/hoseung/Work/pyclusterevol/galaxy/galaxy.py:135: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m_radial[i] = np.sum(m_sorted[ibins[i]:ibins[i+1]])


galaxy 3 is made 

   lambda calculation done. ID, IDx 3 180491
----------Done---------
------------------ 

loading pickled halo list done:
187 187
Mstar now, min: 2.47e+09 5.00e+09
[Halo.load_info] loading info
[Halo.load_info] nout = 187, base =/home/hoseung/Work/data/10002/
/home/hoseung/Work/data/10002/halo/DM/tree_bricks187
[Halo.load_info] loading info
[Halo.load_info] nout = 187, base =/home/hoseung/Work/data/10002/
(catalogue) # galaxies more massive than 5.00e+09 at nout = 187
 Originally the tree selected: 92
Total set length: 100
length of the original allgal: 100
Before associate_gal_hal,length of the original allhal: 3020
Now, 100
Total # galaxies to analyze 1
# complete-tree galaxies 92
# non-complete-tree galaxies 8
AAAAAAAAAAAAAAAAAAAA [0]
In worker, inds =  0
[rd_GM.rd_gal] fname= /home/hoseung/Work/data/10002/GalaxyMaker/GAL_00187/gal_stars_0000001
!!!! mima vx in gm -2143.80615234 1915.94775391
galaxy 1 is made 

   lambda calculation done. ID, IDx 1 142222
--------

AttributeError: 'list' object has no attribute 'meta'

In [74]:
gals[2] = gals[0]

In [78]:
gals[0][0]

In [79]:
nn = gals

In [80]:
gals = [nn[0][0], nn[1][0], nn[2][0]]

그 아이디가 그 아이디가 아닌가봐..; -> 01605가 이상함. 

In [81]:
plot_gals(gals, npix=60,
         fn_save = "/home/hoseung/Work/data/figs/Odd_gal_plot_3")

/usr/local/lib/python3.5/dist-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


3 !!!!!!!!!!!!!!!!!!!


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:119: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:156: RuntimeWarning: divide by zero encountered in log10


1 !!!!!!!!!!!!!!!!!!!
3 !!!!!!!!!!!!!!!!!!!


/usr/local/lib/python3.5/dist-packages/matplotlib/image.py:371: RuntimeWarning: invalid value encountered in less
  rgba[..., 1] = A < 0  # under data
/usr/local/lib/python3.5/dist-packages/matplotlib/image.py:372: RuntimeWarning: invalid value encountered in greater
  rgba[..., 2] = A > 1  # over data
/usr/local/lib/python3.5/dist-packages/matplotlib/image.py:371: RuntimeWarning: invalid value encountered in less
  rgba[..., 1] = A < 0  # under data
/usr/local/lib/python3.5/dist-packages/matplotlib/image.py:372: RuntimeWarning: invalid value encountered in greater
  rgba[..., 2] = A > 1  # over data
/usr/local/lib/python3.5/dist-packages/matplotlib/image.py:371: RuntimeWarning: invalid value encountered in less
  rgba[..., 1] = A < 0  # under data
/usr/local/lib/python3.5/dist-packages/matplotlib/image.py:372: RuntimeWarning: invalid value encountered in greater
  rgba[..., 2] = A > 1  # over data
/usr/local/lib/python3.5/dist-packages/matplotlib/image.py:371: RuntimeWarning: invalid 

In [70]:
gals

[]